### Expectation Maximization Exercise

- The goal is to build a Gaussian Mixture Model (GMM) using EM algorithm
- Following example of [this blog](https://zhiyzuo.github.io/EM/)

In [1]:
class GMM():
    
    # constrcutor
    def __init__(self, X, k=2):
        from scipy.stats import multivariate_normal as mvn
        self.mvn = mvn
        self.data = X
        self.m, self.n = self.data.shape
        self.k = k # k = number of latent factors (Gaussian models)
    
        self.means = np.random.random((self.k, self.n)) # mean vector for the k normals
        self.sigmas = np.array([np.eye(self.n) for i in range(self.k)]) # covariance matrix for the k normals
        self.phi = np.ones(self.k) / self.k # multi-nomial distribution for k latent-factors. default is 1/k
        self.weights = np.ones((self.m, self.k), dtype=float) # weight vector for the data points
    
    # define log likelihood funciton
    # output is the total log-llikelihood across all j models. using iterative calculation to save space
    def loglikelihood(self):
        loglike = 0
        for i in range(self.m):
            tmp = 0
            for j in range(self.k):
                tmp += self.mvn(self.means[j,:], self.sigmas[j, :, :]).pdf(self.data[i, :]) * self.phi[j]
            loglike += np.log(tmp)
        return loglike
    
    def e_step(self):
        # for each of the j models, calculate the pdf likelihood of each data point under it
        # the likelihood is then normalized across all data points, and becomes the weights
        for j in range(self.k):
            self.weights[:, j] = self.mvn(self.means[j, :], self.sigmas[j, :, :]).pdf(self.data) * self.phi[j]
        self.weights = self.weights / self.weights.sum(axis=1).reshape(-1, 1)
    
    
    def m_step(self):
        for j in range(self.k):
            # update the porbability of j-th model phi[j] to be the mean weights for j-th model
            self.phi[j] = self.weights[:, j].mean()
            # update the mean vector of the j-th model to be the weighted avg of data
            # where weights are likelihood of each data point under the j-th model
            self.means[j,:] = self.weights[:, j].T.dot(self.data) / self.weights[:, j].sum()
            # similarly, update the covariance matrix of the j-th model
            self.sigmas[j, :, :] = (self.weights[:, j].reshape(-1, 1)*(self.data - self.means[j,:])).T.dot(self.weights[:, j].reshape(-1, 1)*(self.data - self.means[j,:]))
            self.sigmas[j, :, :] = self.sigmas[j, :, :] / self.weights[:, j].sum()
            
    # function for training / fitting
    def fit(self, tol = 1e-4):
        n_iters = 0 # initialize iteration counter
        loglike = 1 # initialize log-likelihood
        loglike_prev = 0 # previous value of log-likelihood
        # while loop for training. stops when the diff between iterations is small enough
        while (loglike - loglike_prev > tol):
            loglike_prev = self.loglikelihood()
            self.e_step() # call expectation step
            self.m_step() # call maximization step
            n_iters += 1
            loglike = self.loglikelihood()  # evaluate new log-likelihood
            print(f'iteration {n_iters}: log-likelihood = {loglike}') # log progress
        print(f'terminated at {n_iters}-th iteration. log-likelihood = {loglike}')
        
    def pred_prob(self):
        return self.weights
    
    def pred_label(self):
        return self.weights.argmax(axis=1)

In [2]:
# generate data for testing
np.random.seed(614)
# 20 data points from Guassian 1
X = np.random.multivariate_normal([0, 3], [[0.5, 0], [0, 0.8]], 20)
# 50 data points from Gaussian 2
X = np.vstack((X, np.random.multivariate_normal([20, 10], np.identity(2), 50)))
X.shape

(70, 2)

In [3]:
# model training
gmm = GMM(X)
gmm.fit()

iteration 1: log-likelihood = -321.53231311258446
iteration 2: log-likelihood = -263.3709235830559
iteration 3: log-likelihood = -214.28985098622402
iteration 4: log-likelihood = -214.28985098622383
terminated at 4-th iteration. log-likelihood = -214.28985098622383


In [4]:
gmm.means

array([[ 0.39704838,  3.04389729],
       [20.08875246, 10.03920622]])

In [5]:
gmm.sigmas

array([[[ 0.36977051, -0.13030937],
        [-0.13030937,  0.80782007]],

       [[ 0.54702472, -0.02641154],
        [-0.02641154,  1.06350542]]])

In [6]:
gmm.pred_label()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [7]:
gmm.pred_prob()

array([[1.00000000e+000, 1.66444484e-160],
       [1.00000000e+000, 3.44048537e-158],
       [1.00000000e+000, 9.49195082e-169],
       [1.00000000e+000, 1.72367559e-165],
       [1.00000000e+000, 1.27684644e-165],
       [1.00000000e+000, 6.76409215e-184],
       [1.00000000e+000, 5.71227682e-167],
       [1.00000000e+000, 2.08173793e-161],
       [1.00000000e+000, 1.92604390e-161],
       [1.00000000e+000, 1.09584884e-151],
       [1.00000000e+000, 1.02982340e-161],
       [1.00000000e+000, 6.00884706e-150],
       [1.00000000e+000, 2.22383290e-161],
       [1.00000000e+000, 1.56616931e-179],
       [1.00000000e+000, 9.32314945e-177],
       [1.00000000e+000, 1.47836166e-167],
       [1.00000000e+000, 8.82925131e-181],
       [1.00000000e+000, 9.82048859e-171],
       [1.00000000e+000, 1.49314182e-181],
       [1.00000000e+000, 5.42847809e-170],
       [3.87313997e-250, 1.00000000e+000],
       [1.96779597e-293, 1.00000000e+000],
       [9.74681200e-307, 1.00000000e+000],
       [7.2